In [ ]:
!pip install sentinelhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.6/245.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.4 MB/s eta 0:00:00
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6084 sha256=fbc829b8d55913a91bdcc14a5e246e0fef459695b5d99f1b7482053c8384766c
  Stored in directory: /root/.cache/pip/wheels/2f/a1/c8/543df0e8f5e824c3e92a432e32deb9cd89ae686095ee8cfcbe
Successfully built utm


In [ ]:
import requests

# Define the location coordinates and radius
lat = 37.7749
lon = -122.4194
radius = 1000  # in feet

# Google Maps Static API endpoint and parameters
api_key = 'YOUR_API_KEY_HERE'
url = f'https://maps.googleapis.com/maps/api/staticmap?center={lat},{lon}&zoom=15&size=640x640&radius={radius}&sensor=false&key={api_key}'

# Send the API request and retrieve the image
response = requests.get(url)

# Save the image to a file
with open('satellite_image.png', 'wb') as f:
    f.write(response.content)

print('Satellite image saved to satellite_image.png')

Satellite image saved to satellite_image.png


In [ ]:
import os

if not os.path.exists('Lideta.jpg'):
    raise FileNotFoundError('Satellite image file not found.')

In [ ]:
!pip install Pillow

In [ ]:
import requests
from PIL import Image
import numpy as np

def get_elevation(latitude, longitude):
    """
    Retrieves the elevation of a specified location using the Google Maps Elevation API.

    Args:
        latitude (float): The latitude of the location.
        longitude (float): The longitude of the location.

    Returns:
        float: The elevation of the specified location in meters.
    """
    api_key = 'YOUR_API_KEY_HERE'
    url = f'https://maps.googleapis.com/maps/api/elevation/json?locations={latitude},{longitude}&key={api_key}'

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        elevation = data['results'][0]['elevation']
        return elevation
    except requests.exceptions.RequestException as e:
        print(f'Error retrieving elevation: {e}')
        return None

def analyze_satellite_image(image_path):
    """
    Analyzes a satellite image and retrieves the elevation information for the specified location.

    Args:
        image_path (str): The file path of the satellite image.
    """
    # Load the satellite image
    image = Image.open(satellite_image.png)

    # Convert the image to a NumPy array
    image_data = np.array(image)

    # Perform analysis on the image data
    # Example: Retrieve the center coordinates of the image
    height, width, _ = image_data.shape
    center_x = width // 2
    center_y = height // 2

    # Retrieve the elevation for the center coordinates
    latitude = 37.7749  # Replace with the actual latitude
    longitude = -122.4194  # Replace with the actual longitude
    elevation = get_elevation(latitude, longitude)

    if elevation is not None:
        print(f'The elevation of the specified location is {elevation:.2f} meters.')
    else:
        print('Unable to retrieve elevation information.')

# Example usage
analyze_satellite_image('satellite_image.png')

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter
from scipy.spatial.distance import cdist

def preprocess_image(image_path, output_size=(350, 350)):
    """
    Preprocess a satellite image to ensure consistent resolution and format.

    Args:
        image_path (str): The file path of the satellite image.
        output_size (tuple): The desired output size of the image (width, height).

    Returns:
        numpy.ndarray: The preprocessed satellite image.
    """
    # Load the satellite image
    image = Image.open(image_path)

    # Resize the image to the desired output size
    image = image.resize(output_size)

    # Convert the image to a NumPy array
    image_data = np.array(image)

    return image_data

def create_graph(image_data, sigma=1.0):
    """
    Create an undirected graphical model from the satellite image.

    Args:
        image_data (numpy.ndarray): The preprocessed satellite image.
        sigma (float): The standard deviation of the Gaussian kernel for smoothing.

    Returns:
        numpy.ndarray: The adjacency matrix of the graphical model.
        numpy.ndarray: The node features (e.g., color, texture).
    """
    height, width, channels = image_data.shape

    # Compute node features (e.g., color, texture)
    node_features = np.concatenate([
        gaussian_filter(image_data[:, :, i], sigma) for i in range(channels)
    ], axis=-1)

    # Compute the adjacency matrix
    positions = np.stack(np.meshgrid(np.arange(width), np.arange(height)), axis=-1).reshape(-1, 2)
    distances = cdist(positions, positions)
    adjacency_matrix = np.exp(-distances / sigma)

    return adjacency_matrix, node_features

def detect_environmental_features(adjacency_matrix, node_features):
    """
    Detect relevant environmental features in the satellite image using the graphical model.

    Args:
        adjacency_matrix (numpy.ndarray): The adjacency matrix of the graphical model.
        node_features (numpy.ndarray): The node features (e.g., color, texture).

    Returns:
        numpy.ndarray: The segmentation mask, where each pixel is assigned a label (e.g., 0 for trees, 1 for grass).
    """
    # Implement belief propagation or other inference algorithms to perform segmentation
    # and identify regions corresponding to trees, grass, or other features

    # Example: Simple thresholding based on color features
    segmentation_mask = np.zeros_like(node_features[:, 0], dtype=int)
    segmentation_mask[node_features[:, 0] > 150] = 1  # Assign grass label
    segmentation_mask[node_features[:, 1] > 100] = 0  # Assign tree label

    return segmentation_mask.reshape(image_data.shape[:2])

# Example usage
preprocessed_image = preprocess_image('satellite_image.png')
adjacency_matrix, node_features = create_graph(preprocessed_image)
segmentation_mask = detect_environmental_features(adjacency_matrix, node_features)

# Visualize the segmentation results
plt.figure(figsize=(8, 8))
plt.imshow(segmentation_mask, cmap='viridis')
plt.title('Segmentation of Environmental Features')
plt.show()

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter
from scipy.spatial.distance import cdist

def preprocess_image(image_path, output_size=(350, 350)):
    # Code from the previous example

def create_graph(image_data, sigma=1.0):
    # Code from the previous example

def detect_environmental_features(adjacency_matrix, node_features):
    # Code from the previous example

def draw_boundaries(image_data, segmentation_mask, location):
    """
    Draw color-coded boundaries around the detected forest and grassy regions.

    Args:
        image_data (numpy.ndarray): The preprocessed satellite image.
        segmentation_mask (numpy.ndarray): The segmentation mask, where each pixel is assigned a label (e.g., 0 for trees, 1 for grass).
        location (tuple): The (x, y) coordinates of the specified location.

    Returns:
        numpy.ndarray: The image with the drawn boundaries.
    """
    height, width = image_data.shape[:2]

    # Draw the edge from the specified location to the identified cluster of trees
    tree_mask = (segmentation_mask == 0)
    tree_positions = np.argwhere(tree_mask)
    tree_cluster = tree_positions[np.linalg.norm(tree_positions - location, axis=1).argsort()[0]]

    plt.figure(figsize=(8, 8))
    plt.imshow(image_data)
    plt.plot([location[0], tree_cluster[1]], [location[1], tree_cluster[0]], color='yellow', linewidth=2)

    # Draw the color-coded boundaries around the detected forest regions
    for radius in [200, 1000]:
        forest_mask = tree_mask & (np.linalg.norm(tree_positions - location, axis=1) <= radius)
        forest_contours, _ = cv2.findContours(forest_mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in forest_contours:
            if cv2.contourArea(contour) > 100:
                color = 'blue' if radius <= 200 else 'red'
                cv2.drawContours(image_data, [contour], 0, color, 2)

    # Draw the color-coded boundaries around the detected grassy areas
    grass_mask = (segmentation_mask == 1)
    grass_contours, _ = cv2.findContours(grass_mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in grass_contours:
        if cv2.contourArea(contour) > 100:
            cv2.drawContours(image_data, [contour], 0, 'green', 2)

    return image_data

# Example usage
preprocessed_image = preprocess_image('satellite_image.png')
adjacency_matrix, node_features = create_graph(preprocessed_image)
segmentation_mask = detect_environmental_features(adjacency_matrix, node_features)
image_with_boundaries = draw_boundaries(preprocessed_image, segmentation_mask, (100, 200))

plt.figure(figsize=(8, 8))
plt.imshow(image_with_boundaries)
plt.title('Satellite Image with Environmental Feature Boundaries')
plt.show()

In [ ]:
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 33.4 MB/s eta 0:00:00


In [ ]:
!pip install rasterio[warp]

In [ ]:
from rasterio.warp import reproject

In [ ]:
import rasterio
import numpy as np

# Ensure consistent resolution and format
resampled_image = rasterio.warp.reproject(
    sentinel_image,
    dst_crs=CRS.WGS84,
    dst_nodata=0,
    resampling=rasterio.warp.Resampling.bilinear,
    width=int(bbox.get_size()[0] / 10),
    height=int(bbox.get_size()[1] / 10)
)

NameError: name 'sentinel_image' is not defined

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Create the graph
G = nx.Graph()

# Add nodes and edges based on the image pixels and their spatial relationships
for i in range(resampled_image.shape[0]):
    for j in range(resampled_image.shape[1]):
        G.add_node((i, j), feature=resampled_image[i, j])
        for di in [-1, 0, 1]:
            for dj in [-1, 0, 1]:
                if di != 0 or dj != 0:
                    if 0 <= i + di < resampled_image.shape[0] and 0 <= j + dj < resampled_image.shape[1]:
                        G.add_edge((i, j), (i + di, j + dj))